In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

class FeatureExtractor:
    def  __init__(self):
        print("hello")
        self.model = keras.applications.VGG16(weights='imagenet', include_top=True)
        self.feat_extractor = Model(inputs=self.model.input, outputs=self.model.get_layer("fc2").output)
        
    def extract(self, image_path):
        img = image.load_img(image_path, target_size=self.feat_extractor.input_shape[1:3])
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        features = self.feat_extractor.predict(x)[0]   
        return features
    
    def extract_pca_features(self, features):
        features = np.array(features)    
        pca = PCA(n_components=300)
        pca.fit(features)
        pca_features = pca.transform(features)
        return pca_features,pca

In [ ]:
%cd gdrive/MyDrive/sculptures/
%cd ../

/content/gdrive/MyDrive/sculptures
/content/gdrive/MyDrive


In [ ]:
# import the necessary packages
#from featuresextractor import FeatureExtractor
import argparse
import glob
import os
import time
import random
import os
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


# construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-d", "--dataset", required = True,
# 	help = "Path to the directory that contains the images to be indexed")
# args = vars(ap.parse_args())




# initialize the feature extractor
feat_extractor = FeatureExtractor()



images_path = 'sculptures'
image_extensions = ['.jpg', '.png', '.jpeg']   # case-insensitive (upper/lower doesn't matter)
max_num_images = 10000

images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path) for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]
if max_num_images < len(images):
    images = [images[i] for i in sorted(random.sample(xrange(len(images)), max_num_images))]


tic = time.process_time()

features = []
for i, image_path in enumerate(images):
    if i % 500 == 0:
        toc = time.process_time()
        elap = toc-tic;
        print("analyzing image %d / %d. Time: %4.4f seconds." % (i, len(images),elap))
        tic = time.process_time()
    feat = feat_extractor.extract(image_path)
    features.append(feat)
    
# features = np.array(features)    
# pca = PCA(n_components=300)
# pca.fit(features)
# pca_features = pca.transform(features)

pca_features,pca = feat_extractor.extract_pca_features(features)
   

hello
553467904/553467096 [==============================] - 6s 0us/step
analyzing image 0 / 3170. Time: 0.0002 seconds.
analyzing image 500 / 3170. Time: 537.5972 seconds.
analyzing image 1000 / 3170. Time: 541.1450 seconds.
analyzing image 1500 / 3170. Time: 542.3734 seconds.
analyzing image 2000 / 3170. Time: 540.5688 seconds.
analyzing image 2500 / 3170. Time: 538.7149 seconds.
analyzing image 3000 / 3170. Time: 541.2218 seconds.


In [ ]:
import pickle
pickle.dump([images, pca_features, pca], open('/content/gdrive/My Drive/sculptures.p', 'wb'))

print('finished extracting features for %d images' % len(images))

finished extracting features for 3170 images
